<a href="https://colab.research.google.com/github/yeeeeeun/Jeju_dialect_translation/blob/main/Jeju_dialect_translation_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install accelerate>=0.20.1

In [ ]:
!pip install transformers

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig,PreTrainedTokenizerFast
import torch

model_dir = '/content/drive/MyDrive/NLP/pretrained/'

model = BartForConditionalGeneration.from_pretrained(model_dir)

tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
satoori_text = '나도 이거 들어봤신디'

input_ids = tokenizer(satoori_text, return_tensors='pt').input_ids

translation_ids = model.generate(input_ids, max_length=50, num_beams=4, length_penalty=2.0, early_stopping=True)

translated_text = tokenizer.decode(translation_ids[0], skip_special_tokens=True)

print("번역 결과:", translated_text)

번역 결과: 나도 이거 들어봤었는데


In [ ]:
import nltk.translate.bleu_score as bleu
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
pip install pandas openpyxl xlrd

In [ ]:
import pandas as pd

excel_file_path = '/content/drive/MyDrive/NLP/big/datasets_final.xlsx'

data = pd.read_excel(excel_file_path, sheet_name='test')

print(data.head())

           standard            dialect
0   내 딸도 너네 아빠도 하느냐    내 딸도 너네 아방도 하느냐
1           체대 가겠대요            체대 가켄마씨
2  어 나도 몰랐는데 그런데 얘가  어 나도 몰랐는데 그런데 야이가
3    발이 퉁퉁 부어 오른거에요    발이 퉁퉁 부어 오른거라마씨
4              흔한거네              흔한건게게


In [ ]:
data.tail()

,standard,dialect
351,누가요,누가마씨
352,깜짝 놀라면서,깜짝 놀라멍
353,달려왔어요,달려왔수다
354,막 간호사랑 막 얘기하면서,막 간호사영 막 얘기하멍
355,좋겠다,좋겨


In [ ]:
device = torch.device("cuda:0")

In [ ]:

max_length = data.iloc[:, 0].str.len().max()


longest_sentence = data.loc[data.iloc[:, 0].str.len() == max_length, data.columns[0]].values[0]

print("가장 긴 문장의 글자수:", max_length)
print("가장 긴 문장을 출력함:", longest_sentence)

가장 긴 문장의 글자수: 36
가장 긴 문장을 출력함: 할머니가 들려주는 옛날 이야기는 언제 들어도 재미있고 감동적입니다


In [ ]:
import torch
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    preds = p.predictions
    labels = p.label_ids

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    references = [[label.strip()] for label in decoded_labels]
    predictions = [pred.strip() for pred in decoded_preds]

    from nltk.translate.bleu_score import corpus_bleu
    bleu_score = corpus_bleu(references, predictions)

    return {"bleu_score": bleu_score}

class evalDataset(Dataset):
    def __init__(self, standard, dialect, tokenizer):
        self.standard = standard
        self.dialect = dialect
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.standard)

    def __getitem__(self, idx):
        standard_text = self.standard[idx]
        dialect_text = self.dialect[idx]

        inputs = self.tokenizer(
            text=dialect_text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=130,
            )

        outputs = self.tokenizer(
            text=standard_text,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            max_length=130,
        )

        sample = {
            'input_ids': inputs.input_ids.squeeze(),
            'attention_mask': inputs.attention_mask.squeeze(),
            'labels': outputs.input_ids.squeeze()
        }

        return sample


In [ ]:
# 테스트 데이터 전처리

x = data['standard']
y = data['dialect']

test_dataset = evalDataset(x, y, tokenizer)

In [ ]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/big/test",
    predict_with_generate=True,

)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    compute_metrics=compute_metrics,
    eval_dataset = test_dataset,

)

test_dataset = evalDataset(x, y ,tokenizer)

In [ ]:

results = trainer.evaluate()

print(results)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.07710497826337814, 'eval_bleu_score': 0.7200196746841823, 'eval_runtime': 13.0105, 'eval_samples_per_second': 27.362, 'eval_steps_per_second': 3.459}
